### Importing Important Libraries

#### Steps To Be Followed
1. Importing necessary Libraries
2. Creating S3 bucket 
3. Mapping train And Test Data in S3
4. Mapping The path of the models in S3

In [5]:
import sagemaker
import boto3 #to access awsservices
from sagemaker.amazon.amazon_estimator import get_image_uri #have many algorithm in it
from sagemaker.session import s3_input, Session

In [7]:
bucket_name = 'bankapplication' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

ap-south-1


In [12]:
  
s3 = boto3.resource('s3')
bucket_name = 'bankapplicationofakshay' #set unique name like we do in instagram account
my_region = 'ap-south-1'  # Your desired region

try:
    if my_region == 'ap-south-1':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ', e)

S3 bucket created successfully


In [13]:
# set an output path where the trained model will be saved i.e s3 bucket you created above
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)
#every time you train it will create new version of model in s3 bucket

s3://bankapplicationofakshay/xgboost-as-a-built-in-algo/output


#### Downloading The Dataset And Storing in S3

In [14]:
import pandas as pd
import urllib
#download and save with bankclean.csv name
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

#give a variable to csv
try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
#you can see file downloaded in instances itself above .ipynb file

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [15]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [17]:
### Saving Train And Test Into Buckets
## We start with Train Data
## in sagemaker the first column should be dependent varaible so are doing this
import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
#uploading train csv to s3 bucket
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
#while training model data path should be from s3 bucket
s3_input_train = sagemaker.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [18]:
# Test Data Into Buckets
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

### Building Models Xgboot- Inbuilt Algorithm

In [19]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', 
                          repo_version='1.0-1')
#ImageURIProvider

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [20]:
# initialize hyperparameters
#hyperparameter got from experiments running on local machine to avoid charges 
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [22]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB 
                                          output_path=output_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)
# train_use_spot_instances=True,
#train_max_run=300,
#train_max_wait=600) use these parametr to reduce billing hrs

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [23]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-11-16-10-07-15-052


2023-11-16 10:07:15 Starting - Starting the training job...
2023-11-16 10:07:30 Starting - Preparing the instances for training...
2023-11-16 10:08:14 Downloading - Downloading input data......
2023-11-16 10:08:56 Training - Training image download completed. Training in progress...[2023-11-16 10:09:19.214 ip-10-0-163-153.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node 

### Deploy Machine Learning Model As Endpoints

In [26]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')
#instance count  for multiparallel processing

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-11-16-10-25-39-625
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2023-11-16-10-25-39-625
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2023-11-16-10-25-39-625


-----!

#### Prediction of the Test Data

In [32]:
import numpy as np
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values  # Load the data into an array

# Set up the SageMaker predictor and configure the serializer
xgb_predictor.content_type = 'text/csv'  # Set the data type for inference due to serialization
xgb_predictor.serializer = CSVSerializer()  # Set the serializer type

# Perform prediction
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')  # Predict
predictions_array = np.fromstring(predictions[1:], sep=',')  # Transform the prediction into an array
print(predictions_array.shape)


(12357,)


In [33]:
predictions_array

array([0.05214286, 0.05660191, 0.05096195, ..., 0.03436061, 0.02942475,
       0.03715819])

In [35]:
#taken from aws documentation
#confusion matrix
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



In [36]:
#we have to imrpove by 9% and 34% result with hyperparametr tunning

#### Deleting The Endpoints

In [37]:
#to avoid charges we are deleting end points 
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2023-11-16-10-25-39-625


[{'ResponseMetadata': {'RequestId': '3Z70N0B2ZPHEXK5R',
   'HostId': 'oLa/gcQJkoFpDsPcvPomn+F51xRjWTpwvx6iwCUHzcNHifGIgubTWI2wogF2vPNUPEGllxeZosc=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'oLa/gcQJkoFpDsPcvPomn+F51xRjWTpwvx6iwCUHzcNHifGIgubTWI2wogF2vPNUPEGllxeZosc=',
    'x-amz-request-id': '3Z70N0B2ZPHEXK5R',
    'date': 'Thu, 16 Nov 2023 10:42:57 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-16-10-07-15-052/debug-output/index/000000000/000000000030_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-16-10-07-15-052/profiler-output/system/training_job_end.ts'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2023-11-16-10-07-15-052/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-